In [1]:
import numpy as np

In [73]:
new_snow_cat = ["0-5", "5-10", "10-15", "15-20"]
new_snow_thres = {(0, 5): 0.2, (5, 10): 0.5, (10, 15): 1, (15, 20): 3}

wind_cat = ["0-3", "4-7", "8-10", "10-15", "16-30"]
wind_thres = {(0, 3): 0.2, (3, 7): 1, (7, 10): 2, (10, 15): 0.2, (15, 30): 0.01}


new_snow_region = np.array([[0, 4, 6, 18],
                             [0, 4, 6, 18],
                             [0, 4, 6, 18]])

wind_region = np.array([[0, 4, 12, 18],
                         [4, 0, 18, 6],
                         [18, 12, 6, 0]])

In [74]:
def get_score(a, score_dict):
    for key, value in score_dict.items():
        if key[0] <= a < key[1]:
    #    if a < key:
            return value
            break
    return None

the dict is not sorted and the comparison *less than* is random...

In [75]:
new_snow_region_score = [get_score(a, new_snow_thres) for a in new_snow_region.flatten()]
new_snow_region_score = np.array(new_snow_region_score).reshape(new_snow_region.shape)
print(new_snow_region_score)

[[ 0.2  0.2  0.5  3. ]
 [ 0.2  0.2  0.5  3. ]
 [ 0.2  0.2  0.5  3. ]]


In [76]:
wind_region_score = [get_score(a, wind_thres) for a in wind_region.flatten()]
wind_region_score = np.array(wind_region_score).reshape(wind_region.shape)
print(wind_region_score)

[[ 0.2   1.    0.2   0.01]
 [ 1.    0.2   0.01  1.  ]
 [ 0.01  0.2   1.    0.2 ]]


In [77]:
print(wind_region_score * new_snow_region_score)

[[  4.00000000e-02   2.00000000e-01   1.00000000e-01   3.00000000e-02]
 [  2.00000000e-01   4.00000000e-02   5.00000000e-03   3.00000000e+00]
 [  2.00000000e-03   4.00000000e-02   5.00000000e-01   6.00000000e-01]]
